In [196]:
import re
import uuid

import numpy as np

from src.service.ParseService import ParseService


class SimpleParser(ParseService):
    def __init__(self):
        self.title = ['진동속도(cm/s)', '진동레벨[dB(V)]', '소음[dB(A)]']
        self.other_simple_version = False

    def extract_columns(self, table_list):
        columns = ['구분', '진동속도(cm/s)', '진동레벨[dB(V)]', '소음[dB(A)]', '최저치', '최고치', '최저치', '최고치', '최저치', '최고치', '허용기준', '비고']
        
        if '현장관리기준' in table_list:
            self.other_simple_version = True
            table_list = [i for i in table_list if '현장관리기준' not in i]
            
        return [i for i in table_list if i not in columns and not re.match(r'[^\w\s-]', i)]

    def conversion_error_value(self, non_columns_list):
        conversion_error_list = []
        
        for item in non_columns_list:
            if self.other_simple_version and re.match(r'n/*?t\(.*?\)', item, re.IGNORECASE):
                conversion_error_list.extend([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
            elif not self.other_simple_version and re.match(r'n/*?t', item, re.IGNORECASE) or item == '-':
                conversion_error_list.append(np.nan)
            else:
                conversion_error_list.append(item)

        return conversion_error_list

    def delete_other_value(self, conversion_error_list):
        filtered_list = []
        skip_count, data_count = 0, 0

        for index, item in enumerate(conversion_error_list):
            if skip_count > 0:
                skip_count -= 1
                continue
                
            if re.match(r'\d+\.\d+', str(item)) or item is np.nan:
                filtered_list.append(item)
                data_count += 1
                
                if data_count >= 6:
                    if self.other_simple_version:
                        skip_count += 3
                    else:
                        skip_count += 2
                    data_count = 0
            else:
                filtered_list.append(item)
            # try:
            #     if skip_count > 0:
            #         skip_count -= 1
            #         continue
            #     elif re.match(r'\d+\.\d+cm/s', item, re.IGNORECASE):
            #         # self.other_version이면 skip_count가 2, 아니면 skip_count가 1이다. 
            #         if not re.match(r'\d+\.\d+cm/s', conversion_error_list[index + 1], re.IGNORECASE):
            #             skip_count += 1
            #         else:
            #             skip_count += 2
            #     else:
            #         filtered_list.append(item)
            # except TypeError as e:
            #     filtered_list.append(item)

        return filtered_list

    def classification_by_date(self, filtered_list):
        section_list = []
        current_date_section = []

        for item in filtered_list:
            try:
                if re.match(r'\d+월\d+일', item) and current_date_section:
                    section_list.append(current_date_section)
                    current_date_section = []
                current_date_section.append(item)
            except TypeError as e:
                current_date_section.append(item)

        if current_date_section:
            section_list.append(current_date_section)

        return section_list

    def extract_location(self, classification_list):
        location = []
        data_count = 6

        for items in classification_list:
            for item in items:
                try:
                    if len(location) == 0 and not re.match(r'\d+\.\d+', item) and item is not np.nan and not re.match(r'\d+월\d+일', item):
                        location.append(item)
                    if re.match(r'\d+\.\d+', item) or item is np.nan:
                        data_count -= 1
                    elif data_count == 0 and not re.match(r'\d+월\d+일', item):
                        print(f'장소 아이템 : {item}')
                        location.append(item)
                        data_count = 6
                except TypeError as e:
                    data_count -= 1
        
        print(list(set(location)))

        return sorted(list(set(location)))

    def get_dict(self, classification_list, location_list):
        result_dict = {}
        value_list = []

        date_key, location_key = None, None

        for items in classification_list:
            for index, item in enumerate(items):
                if isinstance(item, float):
                    item = str(item)

                if re.match(r'\d+월\d+일', item):
                    date_key = item
                elif item not in location_list:
                    value_list.append(item)
                elif item in location_list:
                    location_key = item
                    unique_key = str(uuid.uuid4())
                    result_dict[location_key] = {} if location_key not in result_dict else result_dict[location_key]

                    result_dict[location_key][unique_key] = {'일시': date_key}
                    idx1, idx2 = index + 1, index + 2
                    for j in range(len(self.title)):
                        result_dict[location_key][unique_key][f'{self.title[j]} 최저치'] = float(items[idx1])
                        result_dict[location_key][unique_key][f'{self.title[j]} 최고치'] = float(items[idx2])
                        
                        idx1 += 2
                        idx2 += 2
                    idx1, idx2 = 0, 0
                        
        return result_dict




In [197]:
import clipboard

test_case = SimpleParser()
table_list = list(filter(lambda x: x, clipboard.paste().replace("\r\n", " ").split(" ")))

print(table_list)

['▮', '2월1일', '구분', '진동속도(cm/s)', '진동레벨[dB(V)]', '소음[dB(A)]', '현장관리기준', '허용기준', '비고', '최저치', '최고치', '최저치', '최고치', '최저치', '최고치', 'S-oil1', 'NT(TL0.03cm/s이하로감지안됨)', '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil2', 'NT(TL0.03cm/s이하로감지안됨)', '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil3', 'NT(TL0.03cm/s이하로감지안됨)', '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil4', 'NT(TL0.03cm/s이하로감지안됨)', '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil5', 'NT(TL0.03cm/s이하로감지안됨)', '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', '오성기업', '0.0221', '0.0323', '-', '-', '60.0', '61.2', '0.1cm/s80dB(A)', '0.2cm/s80dB(A)', 'OK', '울산종합비즈니스센터', '0.073', '0.156', '62.93', '69.85', '69.44', '78.92', '0.1cm/s80dB(A)', '0.2cm/s80dB(A)', 'OK', '고도화학', '0.0867', '0.1143', '-', '-', '74.8', '75.5', '0.1cm/s80dB(A)', '0.2cm/s80dB(A)', 'OK', '가옥(거남3길34-5)', 'NT(TL0.01cm/s이하로감지안됨)', '0.1cm/s75dB(A)', '0.2cm/s75dB(A)', 'OK', '우사(거남3길34-30)', 'NT(TL0.01cm/s이하로감지안됨)', '0.05cm/s60dB(A)', '0.1cm/s60dB(A)', 'OK', '축사(학남리644-2)', '

In [198]:
# 에러 없음
non_columns_list = test_case.extract_columns(table_list)
print(non_columns_list)

['2월1일', 'S-oil1', 'NT(TL0.03cm/s이하로감지안됨)', '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil2', 'NT(TL0.03cm/s이하로감지안됨)', '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil3', 'NT(TL0.03cm/s이하로감지안됨)', '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil4', 'NT(TL0.03cm/s이하로감지안됨)', '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil5', 'NT(TL0.03cm/s이하로감지안됨)', '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', '오성기업', '0.0221', '0.0323', '-', '-', '60.0', '61.2', '0.1cm/s80dB(A)', '0.2cm/s80dB(A)', 'OK', '울산종합비즈니스센터', '0.073', '0.156', '62.93', '69.85', '69.44', '78.92', '0.1cm/s80dB(A)', '0.2cm/s80dB(A)', 'OK', '고도화학', '0.0867', '0.1143', '-', '-', '74.8', '75.5', '0.1cm/s80dB(A)', '0.2cm/s80dB(A)', 'OK', '가옥(거남3길34-5)', 'NT(TL0.01cm/s이하로감지안됨)', '0.1cm/s75dB(A)', '0.2cm/s75dB(A)', 'OK', '우사(거남3길34-30)', 'NT(TL0.01cm/s이하로감지안됨)', '0.05cm/s60dB(A)', '0.1cm/s60dB(A)', 'OK', '축사(학남리644-2)', 'NT(TL0.01cm/s이하로감지안됨)', '0.05cm/s60dB(A)', '0.1cm/s60dB(A)', 'OK', '옹벽', 'NT(TL0.03cm/s이하로감지안됨)', '-', '-', 'OK', '태성산업

In [199]:
conversion_error_list = test_case.conversion_error_value(non_columns_list)
print(conversion_error_list)

['2월1일', 'S-oil1', nan, nan, nan, nan, nan, nan, '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil2', nan, nan, nan, nan, nan, nan, '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil3', nan, nan, nan, nan, nan, nan, '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil4', nan, nan, nan, nan, nan, nan, '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', 'S-oil5', nan, nan, nan, nan, nan, nan, '0.1cm/s80dB(A)', '0.1cm/s80dB(A)', 'OK', '오성기업', '0.0221', '0.0323', nan, nan, '60.0', '61.2', '0.1cm/s80dB(A)', '0.2cm/s80dB(A)', 'OK', '울산종합비즈니스센터', '0.073', '0.156', '62.93', '69.85', '69.44', '78.92', '0.1cm/s80dB(A)', '0.2cm/s80dB(A)', 'OK', '고도화학', '0.0867', '0.1143', nan, nan, '74.8', '75.5', '0.1cm/s80dB(A)', '0.2cm/s80dB(A)', 'OK', '가옥(거남3길34-5)', nan, nan, nan, nan, nan, nan, '0.1cm/s75dB(A)', '0.2cm/s75dB(A)', 'OK', '우사(거남3길34-30)', nan, nan, nan, nan, nan, nan, '0.05cm/s60dB(A)', '0.1cm/s60dB(A)', 'OK', '축사(학남리644-2)', nan, nan, nan, nan, nan, nan, '0.05cm/s60dB(A)', '0.1cm/s60dB(A)', 'OK', '옹벽', nan, n

In [200]:
filtered_list = test_case.delete_other_value(conversion_error_list)
print(filtered_list)

['2월1일', 'S-oil1', nan, nan, nan, nan, nan, nan, 'S-oil2', nan, nan, nan, nan, nan, nan, 'S-oil3', nan, nan, nan, nan, nan, nan, 'S-oil4', nan, nan, nan, nan, nan, nan, 'S-oil5', nan, nan, nan, nan, nan, nan, '오성기업', '0.0221', '0.0323', nan, nan, '60.0', '61.2', '울산종합비즈니스센터', '0.073', '0.156', '62.93', '69.85', '69.44', '78.92', '고도화학', '0.0867', '0.1143', nan, nan, '74.8', '75.5', '가옥(거남3길34-5)', nan, nan, nan, nan, nan, nan, '우사(거남3길34-30)', nan, nan, nan, nan, nan, nan, '축사(학남리644-2)', nan, nan, nan, nan, nan, nan, '옹벽', nan, nan, nan, nan, nan, nan, '태성산업', '0.096', '0.161', '63.81', '70.92', '70.76', '78.21', '부승화학', '0.0497', '0.0591', nan, nan, '63.2', '65.6', '2월2일', 'S-oil1', nan, nan, nan, nan, nan, nan, 'S-oil2', nan, nan, nan, nan, nan, nan, 'S-oil3', nan, nan, nan, nan, nan, nan, 'S-oil4', nan, nan, nan, nan, nan, nan, 'S-oil5', nan, nan, nan, nan, nan, nan, '오성기업', nan, nan, nan, nan, nan, nan, '울산종합비즈니스센터', '0.118', '0.162', '66.27', '70.58', '71.28', '75.06', '고도화학', na

In [201]:
classification_list = test_case.classification_by_date(filtered_list)
print(classification_list)

# 여기까지 문제 없음

[['2월1일', 'S-oil1', nan, nan, nan, nan, nan, nan, 'S-oil2', nan, nan, nan, nan, nan, nan, 'S-oil3', nan, nan, nan, nan, nan, nan, 'S-oil4', nan, nan, nan, nan, nan, nan, 'S-oil5', nan, nan, nan, nan, nan, nan, '오성기업', '0.0221', '0.0323', nan, nan, '60.0', '61.2', '울산종합비즈니스센터', '0.073', '0.156', '62.93', '69.85', '69.44', '78.92', '고도화학', '0.0867', '0.1143', nan, nan, '74.8', '75.5', '가옥(거남3길34-5)', nan, nan, nan, nan, nan, nan, '우사(거남3길34-30)', nan, nan, nan, nan, nan, nan, '축사(학남리644-2)', nan, nan, nan, nan, nan, nan, '옹벽', nan, nan, nan, nan, nan, nan, '태성산업', '0.096', '0.161', '63.81', '70.92', '70.76', '78.21', '부승화학', '0.0497', '0.0591', nan, nan, '63.2', '65.6'], ['2월2일', 'S-oil1', nan, nan, nan, nan, nan, nan, 'S-oil2', nan, nan, nan, nan, nan, nan, 'S-oil3', nan, nan, nan, nan, nan, nan, 'S-oil4', nan, nan, nan, nan, nan, nan, 'S-oil5', nan, nan, nan, nan, nan, nan, '오성기업', nan, nan, nan, nan, nan, nan, '울산종합비즈니스센터', '0.118', '0.162', '66.27', '70.58', '71.28', '75.06', '고도화학',

In [202]:
location_list = test_case.extract_location(classification_list)
print(sorted(location_list))

장소 아이템 : S-oil2
장소 아이템 : S-oil3
장소 아이템 : S-oil4
장소 아이템 : S-oil5
장소 아이템 : 오성기업
장소 아이템 : 울산종합비즈니스센터
장소 아이템 : 고도화학
장소 아이템 : 가옥(거남3길34-5)
장소 아이템 : 우사(거남3길34-30)
장소 아이템 : 축사(학남리644-2)
장소 아이템 : 옹벽
장소 아이템 : 태성산업
장소 아이템 : 부승화학
장소 아이템 : S-oil1
장소 아이템 : S-oil2
장소 아이템 : S-oil3
장소 아이템 : S-oil4
장소 아이템 : S-oil5
장소 아이템 : 오성기업
장소 아이템 : 울산종합비즈니스센터
장소 아이템 : 고도화학
장소 아이템 : 가옥(거남3길34-5)
장소 아이템 : 우사(거남3길34-30)
장소 아이템 : 축사(학남리644-2)
장소 아이템 : 옹벽
장소 아이템 : 태성산업
장소 아이템 : 부승화학
장소 아이템 : S-oil1
장소 아이템 : S-oil2
장소 아이템 : S-oil3
장소 아이템 : S-oil4
장소 아이템 : S-oil5
장소 아이템 : 오성기업
장소 아이템 : 울산종합비즈니스센터
장소 아이템 : 고도화학
장소 아이템 : 가옥(거남3길34-5)
장소 아이템 : 우사(거남3길34-30)
장소 아이템 : 축사(학남리644-2)
장소 아이템 : 옹벽
장소 아이템 : 태성산업
장소 아이템 : 부승화학
장소 아이템 : S-oil1
장소 아이템 : S-oil2
장소 아이템 : S-oil3
장소 아이템 : S-oil4
장소 아이템 : S-oil5
장소 아이템 : 오성기업
장소 아이템 : 울산종합비즈니스센터
장소 아이템 : 고도화학
장소 아이템 : 가옥(거남3길34-5)
장소 아이템 : 우사(거남3길34-30)
장소 아이템 : 축사(학남리644-2)
장소 아이템 : 옹벽
장소 아이템 : 태성산업
장소 아이템 : 부승화학
장소 아이템 : S-oil1
장소 아이템 : S-oil2
장소 아이템 : S-oil3
장소 아이템 : S-oil4
장소 아이템 : S-o

In [203]:
result_dict = test_case.get_dict(classification_list, location_list)
print(result_dict)

{'S-oil1': {'9c85f970-9570-4996-b4bd-145e42459e96': {'일시': '2월1일', '진동속도(cm/s) 최저치': nan, '진동속도(cm/s) 최고치': nan, '진동레벨[dB(V)] 최저치': nan, '진동레벨[dB(V)] 최고치': nan, '소음[dB(A)] 최저치': nan, '소음[dB(A)] 최고치': nan}, 'da7403c9-9d1b-4bfc-b2a1-cb6e816c6097': {'일시': '2월2일', '진동속도(cm/s) 최저치': nan, '진동속도(cm/s) 최고치': nan, '진동레벨[dB(V)] 최저치': nan, '진동레벨[dB(V)] 최고치': nan, '소음[dB(A)] 최저치': nan, '소음[dB(A)] 최고치': nan}, '58e00222-fdb1-4ece-a74a-aaa4fc7393ce': {'일시': '2월3일', '진동속도(cm/s) 최저치': nan, '진동속도(cm/s) 최고치': nan, '진동레벨[dB(V)] 최저치': nan, '진동레벨[dB(V)] 최고치': nan, '소음[dB(A)] 최저치': nan, '소음[dB(A)] 최고치': nan}, 'e713804e-e99c-47e4-9e5b-7e01c8ace711': {'일시': '2월6일', '진동속도(cm/s) 최저치': nan, '진동속도(cm/s) 최고치': nan, '진동레벨[dB(V)] 최저치': nan, '진동레벨[dB(V)] 최고치': nan, '소음[dB(A)] 최저치': nan, '소음[dB(A)] 최고치': nan}, '152167ee-fce7-46ff-8ede-7f35f2702ec0': {'일시': '2월7일', '진동속도(cm/s) 최저치': nan, '진동속도(cm/s) 최고치': nan, '진동레벨[dB(V)] 최저치': nan, '진동레벨[dB(V)] 최고치': nan, '소음[dB(A)] 최저치': nan, '소음[dB(A)] 최고치': nan}, '8b44964f-7bb2-49c

In [204]:
import pandas as pd

for i in result_dict:
    df = pd.DataFrame.from_dict(result_dict[i]).transpose()
    print(df)

                                         일시 진동속도(cm/s) 최저치 진동속도(cm/s) 최고치  \
9c85f970-9570-4996-b4bd-145e42459e96   2월1일            NaN            NaN   
da7403c9-9d1b-4bfc-b2a1-cb6e816c6097   2월2일            NaN            NaN   
58e00222-fdb1-4ece-a74a-aaa4fc7393ce   2월3일            NaN            NaN   
e713804e-e99c-47e4-9e5b-7e01c8ace711   2월6일            NaN            NaN   
152167ee-fce7-46ff-8ede-7f35f2702ec0   2월7일            NaN            NaN   
8b44964f-7bb2-49cd-b749-775ebf2a2918   2월8일            NaN          0.044   
fc3189df-ff49-497d-94cc-2c4e4163ff38  2월13일          0.035          0.038   
93fd501c-88d9-4357-8a4d-d64d6042a2cd  2월14일          0.042          0.058   
30e41efe-22e5-43d1-9ebb-87209c13b0f6  2월15일            NaN            NaN   
472cdbf7-cc71-407f-95f7-09935607296b  2월16일            NaN            NaN   
8924cd46-241a-4e21-9a5a-d5ea3777b31b  2월17일            NaN          0.098   
86444f02-169e-47cb-86ff-65c5d7ef94bb  2월20일            NaN           0.04   